Lab 1 (Word2Vec) <br/>
**Abiyr Maira** <br/>
**19MD0183** <br/>
**№ 55 on Github list**

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')


START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN'),
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

START_N = rule(
    gram('NOUN'),
)

START_T = rule(
    or_(
        rule(gram('PREP')),
        rule(gram('NOUN'))
    ).optional()
)

KAK = eq('как')

INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
)

INCLUDING_S = or_(
    eq('называют'),
    eq('называется'),
    eq('называть')
)


MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
    )

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    rule(
        eq('—'),
        eq('то'),
        eq('есть'),
        eq(',')
    )
)

MID_N = or_(
    rule(
        eq('есть')
    ),
    rule(
        eq('есть'),
        eq('и'),
        eq('другое'),
        eq('значение'),
        eq('-'),
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NPRO')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('NPRO'),
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
    ),
    rule(
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')        
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()        
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        eq('-'),
        gram('CONJ'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('NPRO'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('PREP'),
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF')
    ),
    rule(
        gram('PRCL'),
        gram('PRTF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADVB'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('NPRO'),
        gram('NOUN').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF').repeatable()
    ),
    rule(
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('ADJF'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('ADJF'),
        gram('ADJF').repeatable(),
    ),
    rule(
        gram('ADJF'),
        gram('INFN'),
        gram('CONJ'),
        gram('INFN'),
    ),
    rule(
        gram('CONJ'),
        gram('NOUN'),
        gram('VERB'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('INFN'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
    ),
    rule(
        gram('PREP'),
        gram('ADJF'),
        gram('VERB'),
        eq('16'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('CONJ'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        eq('л'),
        eq('.'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        eq('-'),
        gram('CONJ'),
        gram('NOUN'),
        gram('CONJ'),
        gram('PRTF'),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('CONJ'),
        gram('ADVB'),
        gram('VERB'),
        gram('ADJF'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN')
    ),
    rule(
        gram('PRTS')
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN'),
        gram('NPRO'),
        gram('NOUN'),
        gram('PREP'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN').repeatable(),
        gram('ADVB'),
        gram('VERB'),
        gram('PREP'),
        gram('NOUN').repeatable(),
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('NOUN'),
        gram('ADJF'),
        gram('ADVB'),
        gram('VERB')
    ),
    rule(
        gram('ADVB'),
        gram('PRTF'),
        gram('NOUN'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('PRTF'),
        gram('PREP'),
        gram('ADJF'),
        gram('NOUN'),
        gram('CONJ'),
        gram('NOUN')
    ),
    rule(
        gram('PREP'),
        gram('NOUN'),
        gram('ADJF'),
        gram('PREP'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJS'),
        eq('-'),
        gram('ADJF'),
        gram('NOUN')
    ),
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(article):
    try:
        HYPONYM = eq(utils.deaccent(article))
        RULE = or_(
          rule(HYPONYM, ATAKJE, START, MID, END),
          rule(HYPONYM, MID, END),
          rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
          rule(START_S, END, KAK, HYPONYM),
          rule(END, INCLUDING, HYPONYM),
          rule(START, MID, END),
          rule(START_N, INCLUDING_S, END),
          rule(HYPONYM, MID, eq('.'), END),
          rule(START_N, MID, END),
          rule(HYPONYM, MID, END, eq(','), END),
          rule(START_T, HYPONYM, MID_N, END),
          rule(HYPONYM, START, MID, END),
          rule(HYPONYM, MID, END, eq(','), END, eq(','), END),
          rule(END, eq(','), END, eq(','), INCLUDING_S, START_N),
          rule(END, END, START_N, KAK, END, eq(','), END),
          rule(END, START_N, KAK, END),
          rule(END, START_N, INCLUDING_S, END),
          rule(HYPONYM, eq(','), END, eq(','), END, eq(';'), END, eq(','), END)
        )
        parser = Parser(RULE) 
        text = utils.deaccent(wikipedia.summary(article))
        text = re.sub(r'\(.+?\)', '', text)
        text = text.lower()
        print(text)
        for idx, match in enumerate(parser.findall(text.lower())):
            k = [_.value for _ in match.tokens]
            print(k)
    except:
        e = sys.exc_info()[0]
        print("Не найдено")

In [44]:
df = pd.read_csv("https://raw.githubusercontent.com/maira1207/NLP/master/lab2/Lab2.csv")
charge = df.loc[df['IN CHARGE NAME'] == "Abiyr Maira"]
charge.head(5)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
1296,NaN,1297,55,Abiyr Maira,ТИГЕЛЬ,КУВШИН,СОСУД,ВМЕСТИЛИЩЕ,МЕСТО,ПРОСТРАНСТВО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,NaN,1298,55,Abiyr Maira,ТИМИН,ОСНОВАНИЕ,НУКЛЕОТИД,ГРУППА,СОВОКУПНОСТЬ,СОЧЕТАНИЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,NaN,1299,55,Abiyr Maira,ТИПОЛОГИЗАЦИЯ,МЕТОД,ПУТЬ,МЕСТО,ИССЛЕДОВАНИЯ,ПОЗНАНИЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,NaN,1300,55,Abiyr Maira,ТИПОРАЗМЕР,СТАНДАРТ,ОБРАЗЕЦ,ИЗДЕЛИЕ,РАЗМЕР,ФОРМА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300,NaN,1301,55,Abiyr Maira,ТИХООКЕАНЕЦ,МОРЯК,РАБОТНИК,МАТРОС,ПРОФЕССИЯ,СПЕЦИАЛИСТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("\n\n")
    except: 
        pass

ТИГЕЛЬ
тигель  — это емкость для нагрева, высушивания, сжигания, обжига или плавления различных материалов. тигли — это неотъемлемая часть металлургического и лабораторного оборудования при литье различных металлов, сплавов, и пр. отличительнои особенностью тиглеи является применение для их конструкции огнеупорных материалов и высокоустоичивых к различным воздеиствиям металлов и сплавов.
тигель имеет обычно коническую  или цилиндрическую форму. разновидностью тиглеи являются также плавильные чашки, плавильные лодочки.
['тигель', '—', 'это', 'емкость', 'для', 'нагрева']
['тигли', '—', 'это', 'неотъемлемая', 'часть', 'металлургического', 'и']



ТИМИН
тимин  —  производное пиримидина, одно из пяти азотистых основании. присутствует во всех живых организмах, где вместе с дезоксирибозои входит в состав нуклеозида тимидина, которыи может фосфорилироваться 1—3 остатками фосфорнои кислоты с образованием нуклеотидов тимидин моно-, ди- или трифосфорнои кислоты . дезоксирибонуклеотиды тимина вход